# Machine learning tests are run here
Some code blocks always have to be run, some can be skipped if only certain tests are of interest.

# Imports (mandatory)
We start with importing the necessary packages.

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

# Make the dataset (mandatory)
Here, we make the dataset that we will use.

In [15]:
# load the features
dftrain = np.loadtxt('features.txt')
# convert to ints
dftrain = dftrain.astype(int)
# load the corresponding labels
y_train = np.loadtxt('labels.txt',str)

# Print the dataset (optional)
See the dataset that we're working with.

In [ ]:
print("Features:", dftrain)
print("Labels:", y_train)

# Model 1 (optional)
This is the first experiment.

'2.6.0'